In [ ]:
"""
    the current threshold for zero-light pixel is <=0.2
    Filter the pixel with night <=0.2 & pop >0 to have populated zero light pixel
    """

In [ ]:
import os
import rasterio
from rasterio.warp import reproject, Resampling
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
path = 'data'
light = rasterio.open(f'{path}/NightTimeRasters_100.tif',mode='r') 
pop = rasterio.open(f'{path}/Grid3.tif',mode='r') 

In [ ]:
night_band = light.read(1)
pop_band = pop.read(1)

In [ ]:
night_band = np.argwhere(night_band)
night_band_df = pd.DataFrame(night_band)
night_band_df.columns = ['px','py']

In [ ]:
pop_band = np.argwhere(pop_band)
pop_band_df = pd.DataFrame(pop_band)
pop_band_df.columns = ['px','py']

In [ ]:
night_pop = pop_band_df.merge(night_band_df, how= 'inner')

In [ ]:
lon = []
lat = []
for px, py in zip(night_pop['px'], night_pop['py']):
    """
    convert pixel index to coordinate
    """
    with rasterio.open('nightTimeRasters_100.tif') as map_layer:
        lo = map_layer.xy(px, py)[0]
        lon.append(lo)
        la = map_layer.xy(px, py)[1]
        lat.append(la)

In [ ]:
longitude = pd.DataFrame(lon)
longitude.columns = ['lon']
lattitude = pd.DataFrame(lat)
lattitude.columns = ['lat']
lon_lat = longitude.merge(lattitude, how = 'outer', left_index=True, right_index=True)
night_pop = night_pop.merge(lon_lat, how = 'outer', left_index=True, right_index=True)

In [ ]:
def from_pixel_to_pop(px,py):
        """
        population from pixel coord
        """
    return pop_band[px,py]
def from_pixel_to_light(px, py):
    """
    light from pixel coord
    """
    return night_band[px, py]

In [ ]:
pop = []
light = []
for px, py in zip(night_pop['px'], night_pop['py']):
    p = from_pixel_to_pop(px, py)
    l = from_pixel_to_light(px,py)
    pop.append(p)
    light.append(l)

In [ ]:
population = pd.DataFrame(pop)
population.columns = ['pop']
lightnight = pd.DataFrame(light)
lightnight.columns = ['light']

In [ ]:
night_pop = night_pop.merge(population, how = 'outer', left_index=True, right_index=True)
night_pop = night_pop.merge(lightnight, how = 'outer', left_index=True, right_index=True)

In [ ]:
night_pop['coord'] = night_pop['lat'].apply(lambda x: str(x))+","+night_pop['lon'].apply(lambda x: str(x))

In [ ]:
night_pop.to_csv(f'{path}/population and light.csv',index=False)